In [ ]:
!pip install arize-phoenix langchain openai

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import openai
import phoenix as px
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from phoenix.experimental.callbacks.langchain_tracer import OpenInferenceTracer
from phoenix.trace.trace_dataset import TraceDataset
from tqdm import tqdm

In [ ]:
openai_api_key = getpass("Enter your OpenAI API key")
os.environ["OPENAI_API_KEY"] = openai_api_key
openai.api_key = openai_api_key

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = Chroma(
    persist_directory="/Users/mikeldking/langchain-chroma-arize-docs", embedding_function=embeddings
)
chat_model_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=chat_model_name)
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
)
tracer = OpenInferenceTracer()

In [ ]:
url = "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])
queries

In [ ]:
for query in tqdm(queries):
    chain.run(query, callbacks=[tracer])

In [ ]:
ds = TraceDataset.from_spans(tracer.span_buffer)
px.launch_app(trace=ds)